<a href="https://colab.research.google.com/github/akashsikarwar2211/secure-file-system-using-hybrid-cryptography/blob/main/cryptography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Secure File System using Hybrid Cryptography

from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
import os, json

# Directories
os.makedirs("encrypted_files", exist_ok=True)
os.makedirs("keys", exist_ok=True)
os.makedirs("meta", exist_ok=True)

# Step 1: Generate RSA Key Pair
def generate_keys(username):
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()

    with open(f"keys/{username}_priv.pem", "wb") as priv_file:
        priv_file.write(private_key)
    with open(f"keys/{username}_pub.pem", "wb") as pub_file:
        pub_file.write(public_key)
    print("Keys generated and saved.")

# Step 2 & 3: Encrypt File with AES, Encrypt AES Key with RSA

def encrypt_file(filepath, username):
    # Load public key
    with open(f"keys/{username}_pub.pem", "rb") as pub_file:
        public_key = RSA.import_key(pub_file.read())

    # Generate AES key
    aes_key = get_random_bytes(32)  # 256-bit AES key

    # Read file content
    with open(filepath, "rb") as f:
        file_data = f.read()

    # AES encryption
    cipher_aes = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(file_data)

    # RSA encryption of AES key
    cipher_rsa = PKCS1_OAEP.new(public_key)
    enc_aes_key = cipher_rsa.encrypt(aes_key)

    # Save encrypted file
    filename = os.path.basename(filepath)
    with open(f"encrypted_files/{filename}.enc", "wb") as ef:
        ef.write(ciphertext)

    # Save metadata
    metadata = {
        "nonce": cipher_aes.nonce.hex(),
        "tag": tag.hex(),
        "enc_key": enc_aes_key.hex()
    }
    with open(f"meta/{filename}.json", "w") as meta_file:
        json.dump(metadata, meta_file)

    print("File encrypted and saved.")

# Step 4: Decrypt the File

def decrypt_file(encrypted_filename, username):
    # Load private key
    with open(f"keys/{username}_priv.pem", "rb") as priv_file:
        private_key = RSA.import_key(priv_file.read())

    # Load metadata
    with open(f"meta/{encrypted_filename}.json", "r") as meta_file:
        meta = json.load(meta_file)

    # Convert hex to bytes
    nonce = bytes.fromhex(meta["nonce"])
    tag = bytes.fromhex(meta["tag"])
    enc_key = bytes.fromhex(meta["enc_key"])

    # Decrypt AES key
    cipher_rsa = PKCS1_OAEP.new(private_key)
    aes_key = cipher_rsa.decrypt(enc_key)

    # Read encrypted file
    with open(f"encrypted_files/{encrypted_filename}.enc", "rb") as ef:
        ciphertext = ef.read()

    # AES decryption
    cipher_aes = AES.new(aes_key, AES.MODE_EAX, nonce)
    plaintext = cipher_aes.decrypt_and_verify(ciphertext, tag)

    # Save decrypted file
    with open(f"decrypted_{encrypted_filename}", "wb") as df:
        df.write(plaintext)

    print("File decrypted and saved.")

# Example usage:
# generate_keys("alice")
# encrypt_file("sample.txt", "alice")
# decrypt_file("sample.txt", "alice")


In [ ]:
pip install pycryptodome


In [ ]:
from Crypto.PublicKey import RSA
import os

os.makedirs("keys", exist_ok=True)

def generate_keys(username):
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()

    with open(f"keys/{username}_priv.pem", "wb") as priv_file:
        priv_file.write(private_key)
    with open(f"keys/{username}_pub.pem", "wb") as pub_file:
        pub_file.write(public_key)

    print(f"Keys generated for {username}.")


In [ ]:
import shutil

# Replace 'env_variables.txt' with your actual file name
shutil.move("sample_file.txt", ".env")

print("Converted .txt to .env successfully.")


Converted .txt to .env successfully.


In [ ]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
import json

os.makedirs("encrypted_files", exist_ok=True)
os.makedirs("meta", exist_ok=True)

def encrypt_file(filepath, username):
    with open(f"keys/{username}_pub.pem", "rb") as pub_file:
        public_key = RSA.import_key(pub_file.read())

    aes_key = get_random_bytes(32)
    with open(filepath, "rb") as f:
        file_data = f.read()

    cipher_aes = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(file_data)

    cipher_rsa = PKCS1_OAEP.new(public_key)
    enc_aes_key = cipher_rsa.encrypt(aes_key)

    filename = os.path.basename(filepath)
    with open(f"encrypted_files/{filename}.enc", "wb") as ef:
        ef.write(ciphertext)

    metadata = {
        "nonce": cipher_aes.nonce.hex(),
        "tag": tag.hex(),
        "enc_key": enc_aes_key.hex()
    }
    with open(f"meta/{filename}.json", "w") as meta_file:
        json.dump(metadata, meta_file)

    print(f"File '{filename}' encrypted.")


In [ ]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")


Saving sample_file.txt to sample_file.txt
Uploaded file: sample_file.txt


In [ ]:
!pip install python-dotenv

from dotenv import load_dotenv
import os

load_dotenv()  # Automatically looks for a file named `.env`

print("SECRET_KEY:", os.getenv("SECRET_KEY"))
print("DEBUG:", os.getenv("DEBUG"))


SECRET_KEY: None
DEBUG: None


In [ ]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")


Saving sample_file.txt to sample_file (3).txt
Uploaded file: sample_file (3).txt


In [ ]:
generate_keys("akash")  # This will generate public and private keys for "akash"


Keys generated for akash.


In [ ]:
encrypt_file("sample_file.txt", "akash")  # Encrypt the file for user "akash"


File 'sample_file.txt' encrypted.


In [ ]:
import os

encrypted_file_path = "encrypted_files/sample_file.txt.enc"
file_size = os.path.getsize(encrypted_file_path)
print(f"Encrypted file size: {file_size} bytes")


Encrypted file size: 107 bytes


In [26]:
with open(encrypted_file_path, "rb") as f:
    encrypted_data = f.read()
    print(encrypted_data[:100].hex())  # Print the first 100 bytes in hex


dae63f14ec81b9e3591c7f645973dde3ed07e8b8bb91fc251922cdeb485e78feb01dd6cec5a9d0ec93d285f2afb51039ba26a77c1c88b2e34702d5cf58b8dd6718e51fdf169f0edb05f64ce0682b2ee0d05457323de9f65d8e41c32d01b783274a2cbc83


In [27]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [28]:
!git config --global user.name "Akash Sikarwar"
!git config --global user.email "akash.sikarwar2807@gmail.com"


In [29]:
!git clone https://github.com/akashsikarwar2211/secure-file-system-using-hybrid-cryptography


Cloning into 'secure-file-system-using-hybrid-cryptography'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [30]:
%cd secure-file-system-using-hybrid-cryptography


/content/secure-file-system-using-hybrid-cryptography


In [31]:
%cd cryptography.ipynb


[Errno 2] No such file or directory: 'cryptography.ipynb'
/content/secure-file-system-using-hybrid-cryptography


In [32]:
!cp /path/to/your/files/* .  # This copies files from the location in Colab to the repository folder


cp: cannot stat '/path/to/your/files/*': No such file or directory
